In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import seaborn as sns

In [3]:
path = '../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'
folder_list = [x for x in os.listdir(path) if '.' not in x]
ds = []
for n,x in enumerate(folder_list):
    for i in os.listdir(path+'/'+x+'/'+x):
        ds.append([path+'/'+x+'/'+x+'/'+i,n,x])

df = pd.DataFrame(ds,columns=['Path','Label','Name'])
df.head()

In [4]:
# print(ds)

In [5]:
classes = df.Name.value_counts().index
classes

In [6]:
# sns.set_theme(style="whitegrid")
# plt.figure(figsize=(10,8))
# ax = sns.countplot(y=df.Name)
# ax.set_title('No. of Image');

In [7]:
import random
fig, axs = plt.subplots(3,3,figsize=(15,10))
for i,x in enumerate(df.Name.value_counts().index):
    img = plt.imread(random.choice(df.loc[df.Name==df.Name.value_counts().index[i]].Path.to_list()))
    axs[int((i-i%3)/3),i%3].imshow(img)
    axs[int((i-i%3)/3),i%3].axis('off')
    axs[int((i-i%3)/3),i%3].set_title(x)

In [8]:
# Building Image Pipeline Referenced:
# https://www.kaggle.com/namtrn021200/fish-classification-tensorflow-input-pipeline

dataset = tf.data.Dataset.from_tensor_slices((df.Path,df.Label))
def get_train_val_test_split(ds,shuffle=True,shuffle_size=5000):
    ds_size=len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size,seed=12)
    train_size=int(ds_size*.8)
    val_size=int(ds_size*.1)
    train_ds=ds.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    return train_ds, val_ds, test_ds
train_ds, val_ds, test_ds = get_train_val_test_split(dataset)

In [9]:
# for x in train_ds:
#     print(x)

In [10]:
def load_images(filename,label):
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.convert_image_dtype(image,tf.float32)
    image = tf.image.resize(image,(224,224))
    return image,label
def augment(image,label):
    image = tf.image.random_brightness(image,max_delta=.2)
    image = tf.image.random_flip_left_right(image)
    iamge = tf.image.random_flip_up_down(image)
    image = tf.image.rot90(image)
    image = tf.image.stateless_random_jpeg_quality(image,min_jpeg_quality=90,
                                                  max_jpeg_quality=100,
                                                  seed=(np.random.randint(100),
                                                       np.random.randint(100)))
    return image,label

In [11]:
BATCH_SIZE=32
train_ds = (train_ds
    .shuffle(len(train_ds)*100,seed=42)
    .map(load_images)
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE))
test_ds =(test_ds
         .shuffle(len(test_ds)*100)
         .map(load_images)
         .cache()
         .batch(BATCH_SIZE)
         .prefetch(tf.data.AUTOTUNE))
val_ds =(val_ds
         .shuffle(len(test_ds)*100)
         .map(load_images)
         .cache()
         .batch(BATCH_SIZE)
         .prefetch(tf.data.AUTOTUNE))
batch = next(iter(train_ds))

In [12]:
import tensorflow_hub as hub

resnet_url = 'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5'
feature_extractor_layer = hub.KerasLayer(resnet_url,
                                        trainable=False,
                                        name='feature_extraction_layer',
                                        input_shape=(224,224,3))
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                                              patience=3,
                                                              verbose=1,
                                                              factor=.5,
                                                              min_lr=.00001)
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(9,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
history = model.fit(train_ds,
                 epochs=20,
                 validation_data=val_ds,
                 callbacks=[learning_rate_reduction])

In [13]:
#  import seaborn as sns
# def plot_history(history):
#     fig,ax = plt.subplots(1,2,figsize=(16,6))
#     history_data = pd.DataFrame(history.history)
#     sns.lineplot(data=history_data[['loss','val_loss']],ax=ax[0])
#     sns.lineplot(data=history_data[['accuracy','val_accuracy']],ax=ax[1])
#     ax[0].set_xlabel('Epochs')
#     ax[1].set_xlabel('Epochs');
# plot_history(history)

In [14]:
from sklearn.metrics import classification_report
test_data = [label.numpy() for example, label in test_ds]
y_test = np.concatenate(np.array(test_data))
y_test_class = [classes[x] for x in y_test]
y_pred = tf.argmax(model.predict(test_ds),axis=1)
y_pred_class = [classes[x] for x in y_pred]


In [15]:
# singleImgPath = ../input/a-large-scale-fish-dataset/NA_Fish_Dataset/Black Sea Sprat/00001.png
singleImage = plt.imread('../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset/Gilt-Head Bream/Gilt-Head Bream/00001.png')

sampleImg = (train_ds
    .shuffle(len(train_ds)*100,seed=42)
    .map(load_images)
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE))

# print(df.loc[df.Name==df.Name.value_counts().index[i]].Path.to_list())
# plt.imread(df.loc[df.Name==df.Name.value_counts().index[i]].Path.to_list())
# ax.imshow(singleImage)
# plt.title('matplotlib.pyplot.imread() function Example', fontweight ="bold")
# plt.show()

## from sklearn.metrics import classification_report
test_data = [label.numpy() for example, label in test_ds]
y_test = np.concatenate(np.array(test_data))
y_test_class = [classes[x] for x in y_test]
y_pred = tf.argmax(model.predict(test_ds),axis=1)
y_pred_class = [classes[x] for x in y_pred]
print(classification_report(y_test_class,y_pred_class))

In [ ]:
path_img = '../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset/Black Sea Sprat/Black Sea Sprat/00001.png'
ds_img = [[path_img,-1,'Temp']]
df_img = pd.DataFrame(ds_img,columns=['Path','Label','Name'])
image_set = tf.data.Dataset.from_tensor_slices((df_img.Path,df_img.Label))

In [ ]:
# for x in image_set:
#     print(x)

In [ ]:
image_set =(image_set
         .shuffle(len(image_set)*100)
         .map(load_images)
         .cache()
         .batch(BATCH_SIZE)
         .prefetch(tf.data.AUTOTUNE))

In [ ]:
#  tf.argmax(model.predict(test_ds),axis=1)
arr = model.predict(image_set)
arr=arr[0]


In [ ]:
max_val=0
length=len(arr)

for x in range(length): 
     if(arr[x] > arr[max_val]):
         max_val=x
        
print(classes[max_val])
    



In [ ]:
KERAS_MODEL_NAME="fish_model_file.h5"

In [ ]:
model.save(KERAS_MODEL_NAME)

In [ ]:
TF_LITE_MODEL_FILE_NAME="tf_lite_model_new.tflite"
 

In [ ]:
tf_lite_converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=tf_lite_converter.convert()

In [ ]:
tflite_model_name=TF_LITE_MODEL_FILE_NAME
open(tflite_model_name,"wb").write(tflite_model)